In [ ]:
# Copyright 2026 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Model Garden - Stable Diffusion V2.1 Deployment

<table><tbody><tr>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fcommunity%2Fmodel_garden%2Fmodel_garden_pytorch_sd_2_1_deployment.ipynb">
      <img alt="Google Cloud Colab Enterprise logo" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" width="32px"><br> Run in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_pytorch_sd_2_1_deployment.ipynb">
      <img alt="GitHub logo" src="https://github.githubassets.com/assets/GitHub-Mark-ea2971cee799.png" width="32px"><br> View on GitHub
    </a>
  </td>
</tr></tbody></table>

## Overview

This notebook demonstrates deploying the Stable Diffusion [stabilityai/stable-diffusion-2-1](https://huggingface.co/stabilityai/stable-diffusion-2-1)  model on Vertex AI for online prediction.

### Objective

- Deploy the model to a [Vertex AI Endpoint resource](https://cloud.google.com/vertex-ai/docs/predictions/using-private-endpoints).
- Run online predictions for text-to-image or text-guided-image-to-image.

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing), [Cloud Storage pricing](https://cloud.google.com/storage/pricing), and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

## Before you begin

In [ ]:
# @title Setup Google Cloud project

# @markdown 1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

# @markdown 2. **[Optional]** [Create a Cloud Storage bucket](https://cloud.google.com/storage/docs/creating-buckets) for storing experiment outputs. Set the BUCKET_URI for the experiment environment. The specified Cloud Storage bucket (`BUCKET_URI`) should be located in the same region as where the notebook was launched. Note that a multi-region bucket (eg. "us") is not considered a match for a single region covered by the multi-region range (eg. "us-central1"). If not set, a unique GCS bucket will be created instead.

BUCKET_URI = "gs://"  # @param {type:"string"}

# @markdown 3. **[Optional]** Set region. If not set, the region will be set automatically according to Colab Enterprise environment.

REGION = ""  # @param {type:"string"}

# @markdown 4. If you want to run predictions with A100 80GB or H100 GPUs, we recommend using the regions listed below. **NOTE:** Make sure you have associated quota in selected regions. Click the links to see your current quota for each GPU type: [Nvidia A100 80GB](https://console.cloud.google.com/iam-admin/quotas?metric=aiplatform.googleapis.com%2Fcustom_model_serving_nvidia_a100_80gb_gpus), [Nvidia H100 80GB](https://console.cloud.google.com/iam-admin/quotas?metric=aiplatform.googleapis.com%2Fcustom_model_serving_nvidia_h100_gpus). You can request for quota following the instructions at ["Request a higher quota"](https://cloud.google.com/docs/quota/view-manage#requesting_higher_quota).

# @markdown | Machine Type | Accelerator Type | Recommended Regions |
# @markdown | ----------- | ----------- | ----------- |
# @markdown | a2-ultragpu-1g | 1 NVIDIA_A100_80GB | us-central1, us-east4, europe-west4, asia-southeast1, us-east4 |
# @markdown | a3-highgpu-2g | 2 NVIDIA_H100_80GB | us-west1, asia-southeast1, europe-west4 |
# @markdown | a3-highgpu-4g | 4 NVIDIA_H100_80GB | us-west1, asia-southeast1, europe-west4 |
# @markdown | a3-highgpu-8g | 8 NVIDIA_H100_80GB | us-central1, europe-west4, us-west1, asia-southeast1 |

import datetime
import importlib
import math
import os
import uuid

from google.cloud import aiplatform

! git clone https://github.com/GoogleCloudPlatform/vertex-ai-samples.git

common_util = importlib.import_module(
    "vertex-ai-samples.notebooks.community.model_garden.docker_source_codes.notebook_util.common_util"
)


# Get the default cloud project id.
PROJECT_ID = os.environ["GOOGLE_CLOUD_PROJECT"]

# Get the default region for launching jobs.
if not REGION:
    if not os.environ.get("GOOGLE_CLOUD_REGION"):
        raise ValueError(
            "REGION must be set. See"
            " https://cloud.google.com/vertex-ai/docs/general/locations for"
            " available cloud locations."
        )
    REGION = os.environ["GOOGLE_CLOUD_REGION"]

# Enable the Vertex AI API and Compute Engine API, if not already.
print("Enabling Vertex AI API and Compute Engine API.")
! gcloud services enable aiplatform.googleapis.com compute.googleapis.com

# Cloud Storage bucket for storing the experiment artifacts.
# A unique GCS bucket will be created for the purpose of this notebook. If you
# prefer using your own GCS bucket, change the value yourself below.
now = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
BUCKET_NAME = "/".join(BUCKET_URI.split("/")[:3])

if BUCKET_URI is None or BUCKET_URI.strip() == "" or BUCKET_URI == "gs://":
    BUCKET_URI = f"gs://{PROJECT_ID}-tmp-{now}-{str(uuid.uuid4())[:4]}"
    BUCKET_NAME = "/".join(BUCKET_URI.split("/")[:3])
    ! gsutil mb -l {REGION} {BUCKET_URI}
else:
    assert BUCKET_URI.startswith("gs://"), "BUCKET_URI must start with `gs://`."
    shell_output = ! gsutil ls -Lb {BUCKET_NAME} | grep "Location constraint:" | sed "s/Location constraint://"
    bucket_region = shell_output[0].strip().lower()
    if bucket_region != REGION:
        raise ValueError(
            "Bucket region %s is different from notebook region %s"
            % (bucket_region, REGION)
        )
print(f"Using this GCS Bucket: {BUCKET_URI}")

STAGING_BUCKET = os.path.join(BUCKET_URI, "temporal")
MODEL_BUCKET = os.path.join(BUCKET_URI, "stable_diffusion_v2_1")


# Initialize Vertex AI API.
print("Initializing Vertex AI API.")
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=STAGING_BUCKET)

# Gets the default SERVICE_ACCOUNT.
shell_output = ! gcloud projects describe $PROJECT_ID
project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"
print("Using this default Service Account:", SERVICE_ACCOUNT)


# Provision permissions to the SERVICE_ACCOUNT with the GCS bucket
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.admin $BUCKET_NAME

! gcloud config set project $PROJECT_ID
! gcloud projects add-iam-policy-binding --no-user-output-enabled {PROJECT_ID} --member=serviceAccount:{SERVICE_ACCOUNT} --role="roles/storage.admin"
! gcloud projects add-iam-policy-binding --no-user-output-enabled {PROJECT_ID} --member=serviceAccount:{SERVICE_ACCOUNT} --role="roles/aiplatform.user"

models, endpoints = {}, {}

# The pre-built serving docker image. It contains serving scripts and models.
TEXT_TO_IMAGE_DOCKER_URI = "us-docker.pkg.dev/deeplearning-platform-release/vertex-model-garden/pytorch-inference.cu125.0-4.ubuntu2204.py310"
IMAGE_TO_IMAGE_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-diffusers-serve-opt:20240605_1400_RC00"


def deploy_model(model_id, task, accelerator_type, machine_type, accelerator_count=1):
    """Create a Vertex AI Endpoint and deploy the specified model to the endpoint."""
    common_util.check_quota(
        project_id=PROJECT_ID,
        region=REGION,
        accelerator_type=accelerator_type,
        accelerator_count=accelerator_count,
        is_for_training=False,
    )

    model_name = model_id
    endpoint = aiplatform.Endpoint.create(display_name=f"{model_name}-{task}-endpoint")
    serving_env = {
        "MODEL_ID": model_id,
        "TASK": task,
        "DEPLOY_SOURCE": "notebook",
    }

    if task == "image-to-image":
        model = aiplatform.Model.upload(
            display_name=model_name,
            serving_container_image_uri=IMAGE_TO_IMAGE_DOCKER_URI,
            serving_container_ports=[7080],
            serving_container_predict_route="/predictions/diffusers_serving",
            serving_container_health_route="/ping",
            serving_container_environment_variables=serving_env,
            model_garden_source_model_name="publishers/stability-ai/models/stable-diffusion-2-1",
        )
    else:
        model = aiplatform.Model.upload(
            display_name=model_name,
            serving_container_image_uri=TEXT_TO_IMAGE_DOCKER_URI,
            serving_container_ports=[7080],
            serving_container_predict_route="/predict",
            serving_container_health_route="/health",
            serving_container_environment_variables=serving_env,
            model_garden_source_model_name="publishers/stability-ai/models/stable-diffusion-2-1",
        )

    model.deploy(
        endpoint=endpoint,
        machine_type=machine_type,
        accelerator_type=accelerator_type,
        accelerator_count=accelerator_count,
        deploy_request_timeout=1800,
        service_account=SERVICE_ACCOUNT,
        system_labels={"NOTEBOOK_NAME": "model_garden_pytorch_sd_2_1_deployment.ipynb"},
    )
    print("To load this existing endpoint from a different session:")
    print(
        f'endpoint = aiplatform.Endpoint("projects/{PROJECT_ID}/locations/{REGION}/endpoints/{endpoint.name}")'
    )
    return model, endpoint

## Deploy

In [ ]:
# @title Deploy the Stable Diffusion model to Vertex

# @markdown This section uploads the model to Model Registry and deploys it on the Endpoint. It takes ~15 minutes to finish.
# @markdown Click "Show Code" to see more details.

# @markdown `text-to-image` lets you send text prompts to the endpoint to generate images.

# @markdown `image-to-image` lets you send text prompts and an initial image to the endpoint to
# @markdown condition the generation of new images.


model_id = "stabilityai/stable-diffusion-2-1"

task = "text-to-image"  # @param ["text-to-image", "image-to-image"]
accelerator_type = "NVIDIA_L4"  # @param ["NVIDIA_L4", "NVIDIA_A100_80GB"]

if accelerator_type == "NVIDIA_L4":
    machine_type = "g2-standard-8"
elif accelerator_type == "NVIDIA_A100_80GB":
    machine_type = "a2-ultragpu-1g"
else:
    raise ValueError(f"Unsupported accelerator type: {accelerator_type}")

common_util.check_quota(
    project_id=PROJECT_ID,
    region=REGION,
    accelerator_type=accelerator_type,
    accelerator_count=1,
    is_for_training=False,
)

models["image_model"], endpoints["image_model"] = deploy_model(
    model_id=model_id,
    task=task,
    accelerator_type=accelerator_type,
    machine_type=machine_type,
)
print("endpoint_name:", endpoints["image_model"].name)

# Loads an existing endpoint instance using the endpoint name:
# - Using `endpoint_name = endpoint.name` allows us to get the
#   endpoint name of the endpoint `endpoint` created in the cell
#   above.
# - Alternatively, you can set `endpoint_name = "1234567890123456789"` to load
#   an existing endpoint with the ID 1234567890123456789.
# You may uncomment the code below to load an existing endpoint.

# endpoint_name = ""  # @param {type:"string"}
# aip_endpoint_name = (
#     f"projects/{PROJECT_ID}/locations/{REGION}/endpoints/{endpoint_name}"
# )
# endpoint = aiplatform.Endpoint(aip_endpoint_name)

# print("To load this existing endpoint from a different session:")
# print(
#     f'endpoint = aiplatform.Endpoint("projects/{PROJECT_ID}/locations/{REGION}/endpoints/{endpoint.name}")'
# )

## Predict

In [ ]:
# @title Predict (text-to-image)

# @markdown This section is only for sending predictions to an endpoint with the task `text-to-image`.

# @markdown Once deployment succeeds, you can generate images by sending text prompts to the endpoint.

# @markdown You can also batch send prompts by separating them with a comma.
# @markdown You may adjust the parameters below to achieve best image quality.

if task == "text-to-image":
    comma_separated_prompt_list = "A photo of an astronaut riding a horse on mars, A stone castle in a forest by the river"  # @param {type: "string"}
    prompt_list = [x.strip() for x in comma_separated_prompt_list.split(",")]
    negative_prompt = ""  # @param {type: "string"}
    height = 768  # @param {type:"number"}
    width = 768  # @param {type:"number"}
    num_inference_steps = 25  # @param {type:"number"}
    guidance_scale = 7.5  # @param {type:"number"}

    instances = [{"text": prompt} for prompt in prompt_list]
    parameters = {
        "negative_prompt": negative_prompt,
        "height": height,
        "width": width,
        "num_inference_steps": num_inference_steps,
        "guidance_scale": 7.5,
    }

    response = endpoints["image_model"].predict(
        instances=instances, parameters=parameters
    )
    images = [
        common_util.base64_to_image(prediction.get("output"))
        for prediction in response.predictions
    ]
    display(common_util.image_grid(images, rows=math.ceil(len(images) ** 0.5)))
else:
    print(
        "To run `text-to-image` prediction, deploy the model with `text-to-image` task."
    )

In [ ]:
# @title Predict with Dynamic LoRA (text-to-image)
# @markdown You may specify a LoRA along with the request by setting `lora_id`. The LoRA will be loaded dynamically into the base model for the current prediction request. Note that this LoRA will not affect any subsequent requests, unless the same LoRA is specified in the request.

# @markdown `lora_id` should be a Hugging Face id, or a GCS uri (with "gs://" prefix) to the LoRA directory.

# @markdown Example request:

# @markdown ```
# @markdown {
# @markdown   "instances": [{"text": "papercut a red fox"}],
# @markdown   "parameters": {
# @markdown     "lora_id": "TheLastBen/Papercut_SDXL"
# @markdown   }
# @markdown }
# @markdown ```

# Loads an existing endpoint instance using the endpoint name:
# - Using `endpoint_name = endpoint.name` allows us to get the endpoint name of
#   the endpoint `endpoint` created in the cell above.
# - Alternatively, you can set `endpoint_name = "1234567890123456789"` to load
#   an existing endpoint with the ID 1234567890123456789.

# You may uncomment the code below to load an existing endpoint.
# endpoint_name = ""  # @param {type:"string"}
# aip_endpoint_name = (
#     f"projects/{PROJECT_ID}/locations/{REGION}/endpoints/{endpoint_name}"
# )
# endpoint = aiplatform.Endpoint(aip_endpoint_name)
# print("Using this existing endpoint from a different session: {aip_endpoint_name}")

prompt = "A painting from Michel Tolmer"  # @param {type: "string"}
lora_id = "mathieuripert/tolmer-model"  # @param {type: "string"}

if task == "text-to-image":
    instances = [{"text": prompt}]
    parameters = {"lora_id": lora_id}

    response = endpoints["image_model"].predict(
        instances=instances, parameters=parameters
    )
    images = [
        common_util.base64_to_image(prediction.get("output"))
        for prediction in response.predictions
    ]
    display(common_util.image_grid(images, rows=math.ceil(len(images) ** 0.5)))
else:
    print(
        "To run `text-to-image` prediction, deploy the model with `text-to-image` task."
    )

In [ ]:
# @title Predict (text-guided image-to-image)

# @markdown This section is only for sending predictions to an endpoint with the task `image-to-image`.

# @markdown Once deployment succeeds, you can generate images by sending links to images and text prompts to the endpoint.

if task == "image-to-image":
    init_image_url = "https://raw.githubusercontent.com/CompVis/stable-diffusion/main/assets/stable-samples/img2img/sketch-mountains-input.jpg"  # @param {type: "string"}
    prompt = "A fantasy landscape trending on artstation"  # @param {type: "string"}
    negative_prompt = ""  # @param {type: "string"}
    height = 768  # @param {type:"number"}
    width = 768  # @param {type:"number"}
    num_inference_steps = 25  # @param {type:"number"}
    guidance_scale = 7.5  # @param {type:"number"}

    parameters = {
        "negative_prompt": negative_prompt,
        "height": height,
        "width": width,
        "num_inference_steps": num_inference_steps,
        "guidance_scale": 7.5,
    }

    init_image = common_util.download_image(init_image_url)
    display(init_image)

    instances = [
        {
            "prompt": prompt,
            "negative_prompt": negative_prompt,
            "image": common_util.image_to_base64(init_image),
        },
    ]

    response = endpoints["image_model"].predict(
        instances=instances, parameters=parameters
    )
    images = [common_util.base64_to_image(image) for image in response.predictions]
    display(common_util.image_grid(images, rows=math.ceil(len(images) ** 0.5)))
else:
    print(
        "To run `image-to-image` prediction, deploy the model with `image-to-image` task."
    )

## Clean up resources

In [ ]:
# @title Delete the models and endpoints
# @markdown  Delete the experiment models and endpoints to recycle the resources
# @markdown  and avoid unnecessary continuous charges that may incur.

# Undeploy model and delete endpoint.
for endpoint in endpoints.values():
    endpoint.delete(force=True)

# Delete models.
for model in models.values():
    model.delete()

delete_bucket = False  # @param {type:"boolean"}
if delete_bucket:
    ! gsutil -m rm -r $BUCKET_NAME